## Investigate time aspect
Here we investigate the time structure throughout the instrument.

In [ ]:
import mcstasscript as ms
import template

In [ ]:
# Set path to instrument code folder and data placement to keep folder clean
instrument = template.make(include_event_monitors=False, input_path="instrument_code", output_path="data/data")

### Show diagrams with wavelength and time

In [ ]:
instrument.set_parameters(l_min=0.5, l_max=4, n_pulses=2)
instrument.settings(ncount=1E7)

instrument.show_diagram(True, variable="l")

In [ ]:
instrument.show_diagram(True, variable="t")

### Add monitors related to investigation

In [ ]:
# Use clean version of instrument
instrument = template.make(include_event_monitors=False, input_path="instrument_code", output_path="data/data")

def add_tof_monitors(instrument, name, after, tmin=0, tmax=1/6):    
    """ Functions that adds a series of monitors after a given component """

    if isinstance(after, str):
        after = instrument.get_component(after)

    # Some components in the instrument have lengths, check if the after component is a guide
    if after.component_name in ["Guide_gravity", "Elliptic_guide_gravity"]:
        displacement = after.l + 1E-6
    else:
        displacement = 0
    
    lmon = instrument.add_component(name + "_Lmon", "L_monitor", after=after)
    lmon.set_parameters(nL=50, Lmin="l_min", Lmax="l_max", xwidth=0.1, yheight=0.1,
                        filename=f'"{name}_lmon.dat"', restore_neutron=1)
    lmon.set_AT(displacement, RELATIVE=after)

    tofmon = instrument.add_component(name + "_TOFmon", "TOF_monitor", after=lmon)
    tofmon.set_parameters(nt=50, tmin=0, tmax=tmax*1E6, xwidth=0.1, yheight=0.1,
                        filename=f'"{name}_TOFmon.dat"', restore_neutron=1)
    tofmon.set_AT(displacement, RELATIVE=after)

    toflam = instrument.add_component(name + "_TOFLammon", "TOFLambda_monitor", after=lmon)
    toflam.set_parameters(nt=50, tmin=0, tmax=tmax*1E6, 
                          nL=50, Lmin="l_min", Lmax="l_max",
                          xwidth=0.1, yheight=0.1,
                          filename=f'"{name}_TOFLam.dat"', restore_neutron=1)
    toflam.set_AT(displacement, RELATIVE=after)

# Insert monitors at several places
add_tof_monitors(instrument, "after_chopper", after="chopper", tmax=0.015)
add_tof_monitors(instrument, "after_expanding", after="expanding", tmax=0.025)
add_tof_monitors(instrument, "after_focusing", after="focusing", tmax=0.2)

### Run the instrument and plot data

In [ ]:
instrument.settings(suppress_output=True, ncount=1E7)
instrument.set_parameters(n_pulses=1)
data = instrument.backengine()

In [ ]:
ms.make_sub_plot(data)

### Run with larger number of pulses to check overlap

In [ ]:
instrument = template.make(include_event_monitors=False, input_path="instrument_code", output_path="data/data")

# Insert monitors at several places
add_tof_monitors(instrument, "after_chopper", after="chopper", tmax=0.015 + 1/14)
add_tof_monitors(instrument, "after_expanding", after="expanding", tmax=0.025 + 1/14)
add_tof_monitors(instrument, "after_focusing", after="focusing", tmax=0.2 + 1/14)

instrument.settings(suppress_output=True, ncount=1E7)
instrument.set_parameters(n_pulses=2)
data = instrument.backengine()

In [ ]:
ms.make_sub_plot(data)